In [212]:
!git clone https://github.com/Pyroxene889/Store-Sales

fatal: destination path 'Store-Sales' already exists and is not an empty directory.


In [213]:
!pip install opendatasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [214]:
import opendatasets as od
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px

# Data Cleaning

## Data Collecting

In [215]:
od.download(
	"https://www.kaggle.com/competitions/store-sales-time-series-forecasting/data?select=holidays_events.csv", force = True)

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: ikramlaaouane
Your Kaggle Key: ··········


100%|██████████| 21.4M/21.4M [00:01<00:00, 11.5MB/s]



Extracting archive ./store-sales-time-series-forecasting/store-sales-time-series-forecasting.zip to ./store-sales-time-series-forecasting


In [216]:
df_train1 = pd.read_csv('/content/store-sales-time-series-forecasting/train.csv')

In [217]:
df_train1.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


In [218]:
df_test1 = pd.read_csv('/content/store-sales-time-series-forecasting/test.csv')

In [219]:
df_test1.head()

,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2
3,3000891,2017-08-16,1,BEVERAGES,20
4,3000892,2017-08-16,1,BOOKS,0


In [220]:
df_stores = pd.read_csv('/content/store-sales-time-series-forecasting/stores.csv')

In [221]:
df_holidays = pd.read_csv('/content/store-sales-time-series-forecasting/holidays_events.csv')
df_holidays["type"].value_counts().index

Index(['Holiday', 'Event', 'Additional', 'Transfer', 'Bridge', 'Work Day'], dtype='object')

In [222]:
df_oil = pd.read_csv('/content/store-sales-time-series-forecasting/oil.csv')
df_oil.head()

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20


In [223]:
df_transactions = pd.read_csv('/content/store-sales-time-series-forecasting/transactions.csv')

### Clean table oil

In [224]:
def fillna_oil(df):
  df2=df
  df2['oil2']=df2['dcoilwtico'].shift(1)
  df2['oil3']=df2['dcoilwtico'].shift(-1)

  df2.loc[df2['dcoilwtico'].isnull(),'dcoilwtico']=df2['oil2']
  df2.loc[df2['dcoilwtico'].isnull(),'dcoilwtico']=df2['oil3']

  df2.drop(['oil2', 'oil3'], axis=1,inplace=True)
  df2.head()

  df2.head(20)
  return df2

In [225]:
# Clean la table oil
df_oil = fillna_oil(df_oil)

In [226]:
df_oil.head()

,date,dcoilwtico
0,2013-01-01,93.14
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20


In [227]:
df_oil.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1218 entries, 0 to 1217
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        1218 non-null   object 
 1   dcoilwtico  1218 non-null   float64
dtypes: float64(1), object(1)
memory usage: 19.2+ KB


### Merge tous la data en une seule DataFrame

In [228]:
# p=0 si on traite le train data, et 1 si on traite le teste data
def Merge_Data(df, p=0):
  ## Merge train_data avec stores_data
  #df = df.set_index('store_nbr').join(df_stores.set_index('store_nbr'))
  df = df.join(df_stores.set_index('store_nbr'), on='store_nbr')
  ## Merge la data avec holidays_data
  df = df.set_index('date').join(df_holidays.set_index('date'), lsuffix='_store', rsuffix='_holiday')
  ## Merge la data avec oil_data
  df = df.join(df_oil.set_index('date'), on='date')
  ## Merge la data avec transactions pour le train
  if p==0:
    df = pd.merge(df, df_transactions, on=['date', 'store_nbr'])
  return df


Merge Data Train

In [229]:
df_train_merge = Merge_Data(df_train1)
df_train_merge.head()

,date,id,store_nbr,family,sales,onpromotion,city,state,type_store,cluster,type_holiday,locale,locale_name,description,transferred,dcoilwtico,transactions
0,2013-01-01,561,25,AUTOMOTIVE,0.0,0,Salinas,Santa Elena,D,1,Holiday,National,Ecuador,Primer dia del ano,False,93.14,770
1,2013-01-01,562,25,BABY CARE,0.0,0,Salinas,Santa Elena,D,1,Holiday,National,Ecuador,Primer dia del ano,False,93.14,770
2,2013-01-01,563,25,BEAUTY,2.0,0,Salinas,Santa Elena,D,1,Holiday,National,Ecuador,Primer dia del ano,False,93.14,770
3,2013-01-01,564,25,BEVERAGES,810.0,0,Salinas,Santa Elena,D,1,Holiday,National,Ecuador,Primer dia del ano,False,93.14,770
4,2013-01-01,565,25,BOOKS,0.0,0,Salinas,Santa Elena,D,1,Holiday,National,Ecuador,Primer dia del ano,False,93.14,770


Merge Test Data

In [230]:
df_test_merge = Merge_Data(df_test1,1)
df_test_merge.head()

,id,store_nbr,family,onpromotion,city,state,type_store,cluster,type_holiday,locale,locale_name,description,transferred,dcoilwtico
date,,,,,,,,,,,,,,
2017-08-16,3000888,1,AUTOMOTIVE,0,Quito,Pichincha,D,13,NaN,NaN,NaN,NaN,NaN,46.8
2017-08-16,3000889,1,BABY CARE,0,Quito,Pichincha,D,13,NaN,NaN,NaN,NaN,NaN,46.8
2017-08-16,3000890,1,BEAUTY,2,Quito,Pichincha,D,13,NaN,NaN,NaN,NaN,NaN,46.8
2017-08-16,3000891,1,BEVERAGES,20,Quito,Pichincha,D,13,NaN,NaN,NaN,NaN,NaN,46.8
2017-08-16,3000892,1,BOOKS,0,Quito,Pichincha,D,13,NaN,NaN,NaN,NaN,NaN,46.8


### Data Cleaning

In [231]:
df_train_merge.isnull().sum()

date                  0
id                    0
store_nbr             0
family                0
sales                 0
onpromotion           0
city                  0
state                 0
type_store            0
cluster               0
type_holiday    2346168
locale          2346168
locale_name     2346168
description     2346168
transferred     2346168
dcoilwtico       807246
transactions          0
dtype: int64

In [232]:
def Cleaning(df):

  # Transformer les jours fériers qui ont été transférré en un autre jour en jour normal "work Day"
  df.loc[df['transferred'] == True, 'type_holiday'] = 'Work Day'

  #Supprimer la colonne "locale_name", "description" et "transferred", puisqu'on n'en a pas besoin
  df = df.drop(['id'], axis = 1)
  df = df.drop(['locale_name'], axis = 1)
  df = df.drop(['description'], axis = 1)
  df = df.drop(['transferred'], axis = 1)
  # Remplacer les valeurs NaN de la colonne "type_holiday" avec "work"
  df["type_holiday"].fillna("work", inplace = True)
  # Remplacer les valeurs NaN de la colonne "locale" avec "National"
  df["locale"].fillna("National", inplace = True)

  
  # Trier la DataFrame par la colonne 'date' pour s'assurer que les dates sont dans l'ordre
  df = df.sort_values(by='date')


  # Remplacer les valeurs NaN avec la valeur moyenne
  df['dcoilwtico'] = df['dcoilwtico'].interpolate()
  


  return df


In [233]:
df_train_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2805231 entries, 0 to 2805230
Data columns (total 17 columns):
 #   Column        Dtype  
---  ------        -----  
 0   date          object 
 1   id            int64  
 2   store_nbr     int64  
 3   family        object 
 4   sales         float64
 5   onpromotion   int64  
 6   city          object 
 7   state         object 
 8   type_store    object 
 9   cluster       int64  
 10  type_holiday  object 
 11  locale        object 
 12  locale_name   object 
 13  description   object 
 14  transferred   object 
 15  dcoilwtico    float64
 16  transactions  int64  
dtypes: float64(2), int64(5), object(10)
memory usage: 385.2+ MB


In [234]:
df_train_clean = Cleaning(df_train_merge)
df_train_clean.isnull().sum()

date            0
store_nbr       0
family          0
sales           0
onpromotion     0
city            0
state           0
type_store      0
cluster         0
type_holiday    0
locale          0
dcoilwtico      0
transactions    0
dtype: int64

In [235]:
df_train_clean[df_train_clean['date']=='2013-01-05'].head()

,date,store_nbr,family,sales,onpromotion,city,state,type_store,cluster,type_holiday,locale,dcoilwtico,transactions
5597,2013-01-05,41,LAWN AND GARDEN,2.000,0,Machala,El Oro,D,4,Work Day,National,93.120026,1021
5603,2013-01-05,41,PET SUPPLIES,0.000,0,Machala,El Oro,D,4,Work Day,National,93.120053,1021
5602,2013-01-05,41,PERSONAL CARE,178.000,0,Machala,El Oro,D,4,Work Day,National,93.120079,1021
5601,2013-01-05,41,MEATS,144.726,0,Machala,El Oro,D,4,Work Day,National,93.120105,1021
5600,2013-01-05,41,MAGAZINES,0.000,0,Machala,El Oro,D,4,Work Day,National,93.120132,1021


In [236]:
df_test_clean = Cleaning(df_test_merge)
df_test_clean.isnull().sum()

store_nbr       0
family          0
onpromotion     0
city            0
state           0
type_store      0
cluster         0
type_holiday    0
locale          0
dcoilwtico      0
dtype: int64